In [1]:
# LSTM with regression framing
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import pickle

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# fix random seed for reproducibility
numpy.random.seed(7)

# load the dataset
dataframe = pandas.read_csv('data/NL Pb till Dec 2016.csv', usecols=[2,3,4,5,6,7], engine='python', skipfooter=3)

dataset = dataframe.values
dataset = dataset.astype('float32')

Using Theano backend.


In [2]:
# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)

In [3]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = []
        for j in range(look_back):
            a = numpy.concatenate([a, dataset[i + j][:-1]])
        print(a)
        dataX.append(a)
        dataY.append(dataset[i + look_back, 5])
    return numpy.array(dataX), numpy.array(dataY)

# reshape into X=t and Y=t+1
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [5]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [6]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(32, input_shape=(look_back * 5, 1)))
model.add(Dense(40,  activation='relu'))
# model.add(Dense(20,  activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100, batch_size=5, verbose=2)

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/100
2s - loss: 2224512.2099
Epoch 2/100
2s - loss: 1427445.0788
Epoch 3/100
2s - loss: 808197.4487
Epoch 4/100
2s - loss: 634472.1024
Epoch 5/100
2s - loss: 616228.2469
Epoch 6/100
2s - loss: 615213.9655
Epoch 7/100
2s - loss: 613345.5695
Epoch 8/100
2s - loss: 608352.9527
Epoch 9/100
2s - loss: 603192.1278
Epoch 10/100
2s - loss: 598705.1763
Epoch 11/100
2s - loss: 599445.2899
Epoch 12/100
2s - loss: 601555.7603
Epoch 13/100
2s - loss: 595453.3890
Epoch 14/100
2s - loss: 597909.7620
Epoch 15/100
2s - loss: 592719.4519
Epoch 16/100
2s - loss: 589379.9750
Epoch 17/100
2s - loss: 587453.3575
Epoch 18/100
2s - loss: 579290.7761
Epoch 19/100
2s - loss: 577002.5127
Epoch 20/100
2s - loss: 574920.9791
Epoch 21/100
2s - loss: 570568.8492
Epoch 22/100
2s - loss: 567921.0528
Epoch 23/100
2s - loss: 564784.0997
Epoch 24/100
2s - loss: 565348.0474
Epoch 25/100
2s - loss: 561433.5286
Epoch 26/100
2s - loss: 559655.5958
Epoch 27/100
2s - loss: 556957.1070
Epoch 28/100
2s - loss: 554791.3267

In [7]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [8]:
# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
#testPredict = scaler.inverse_transform(testPredict)
#testY = scaler.inverse_transform([testY])

In [9]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 694.27 RMSE


In [10]:
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Test Score: 979.47 RMSE


In [ ]:
'''
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
'''